In [2]:
# Importing standard Qiskit libraries:
import numpy as np     
from qiskit import*# QuantumCircuit, transpile, Aer, IBMQ, execute, assemble
from qiskit.providers.ibmq import least_busy
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from scipy import sparse
from scipy.linalg import sqrtm
# Loading your IBM Quantum account(s):
provider = IBMQ.load_account()

/tmp/ipykernel_61/2868879444.py:11: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  provider = IBMQ.load_account()


# Discrete Brillouin zone

In [3]:
number_of_mesh_points = 4
k_x_lst = list(np.linspace(-np.pi,np.pi,number_of_mesh_points))
k_y_lst = list(np.linspace(-np.pi,np.pi,number_of_mesh_points))
discretized_Brillouin_zone = []
for i in range(number_of_mesh_points):
    for j in range(number_of_mesh_points):
        discretized_Brillouin_zone.append([k_x_lst[i],k_y_lst[j]])
delta_k_x = k_x_lst[1]-k_x_lst[0]
delta_k_y = k_y_lst[1]-k_y_lst[0]        

In [4]:
mu = 0.05

In [5]:
# Number of points = numnumber_of_mesh_points*number_of_mesh_points
#point_index = 0

In [6]:
r"""
# k_x and k_y
k_x,k_y = k_x_lst[point_index],k_y_lst[point_index]
# k+delta_k_x and k+delta_k_y

F_k_lst_of_points = [[(k_x,k_y),(k_x+delta_k_x,k_y)],
[(k_x+delta_k_x,k_y),(k_x+delta_k_x,k_y+delta_k_y)],
[(k_x,k_y+delta_k_y),(k_x+delta_k_x,k_y+delta_k_y)],
[(k_x,k_y),(k_x,k_y+delta_k_y)]]""";

In [7]:
# Index runs from 0 to 3.
#F_k_index = 2

In [8]:
#k_x,k_y = F_k_lst_of_points[F_k_index][0][0],F_k_lst_of_points[F_k_index][0][1]
#k_x_p,k_y_p = F_k_lst_of_points[F_k_index][1][0],F_k_lst_of_points[F_k_index][1][1]

# $U_{3}$ gate

In [9]:
# U_{3} is required for preparing the state |\psi(\theta)>
def U3(theta,lamb,phi):
    return np.array([[np.cos(theta/2),-np.sin(theta/2)*np.exp(1j*lamb)],
                     [np.exp(1j*phi)*np.sin(theta/2),np.exp(1j*(lamb+phi))*np.cos(theta/2)]])

# Defining the two angles $\theta$ and $\phi$

In [10]:
def angle_theta(k_x,k_y):
    E_p = np.sqrt((np.sin(k_x))**2+(np.sin(k_y))**2+(np.cos(k_x)+np.cos(k_y)+mu)**2)
    return np.arccos((np.cos(k_x)+np.cos(k_y)+mu)/E_p)
def angle_phi(k_x,k_y):
    return np.arccos(np.sin(k_y)/np.sqrt((np.sin(k_y))**2+(np.sin(k_x))**2))

# Controlled W gate required for controlled $U_{3}$ operation

In [11]:
def controlled_W(matrix,c,t):
    I = np.identity(2)
    Z = np.matrix([[1,0],[0,-1]])
    X = np.matrix([[0,1],[1,0]])
    PI_0 = (I+Z)/2
    PI_1 = (I-Z)/2    
    Matrices = {'I':I,'PI_0':PI_0,'X':matrix, 'PI_1':PI_1}
    p0 = ['I']*3
    p1 = ['I']*3    
    p0[c] = 'PI_0'
    p1[c] = 'PI_1'
    p1[t] = 'X'
    PI_0_matrix = Matrices[p0[0]]
    for i in range(1,3):
        PI_0_matrix = sparse.kron(Matrices[p0[i]],PI_0_matrix)
    PI_1_matrix = Matrices[p1[0]]
    for i in range(1,3):
        PI_1_matrix = sparse.kron(Matrices[p1[i]],PI_1_matrix)
    return PI_0_matrix+PI_1_matrix

# Constructing the upper band wavefunction

In [12]:
def upper_band_wavefunction(k_x,k_y):
    qr = QuantumRegister(2,"q")
    
    # Create the ancilla register
    anc = QuantumRegister(1,"ancilla")
    # Create the quantum circuit
    qc = QuantumCircuit(anc,qr)  
    
    qc.x(1)
    qc.cx(2,1)
    theta = angle_theta(k_x,k_y)
    phi = angle_phi(k_x,k_y)
    qc.cu(theta,-phi,phi,0,1,2)
    qc.cx(2,1)
    return qc
#upper_band_wavefunction(k_x,k_y).draw()

# Intraband evolution circuit

In [13]:
def intraband_evolution(k_x,k_y,k_x_p,k_y_p):

    # Create the quantum circuit
    qc = QuantumCircuit(3,1)  
    qc.h(0)
    qc = qc.compose(upper_band_wavefunction(k_x,k_y))    
    
    qc.barrier()
    
    qc.ccx(0,2,1)
    # $CCU_{3}^{\dagger}$ gate.
    W = sqrtm(np.matrix(U3(angle_theta(k_x,k_y),-angle_phi(k_x,k_y),angle_phi(k_x,k_y))).getH())
    qc.unitary(controlled_W(W,1,2).A,[0,1,2])
    qc.cx(0,1)
    qc.unitary(np.matrix(controlled_W(W,1,2).A).getH(),[0,1,2])
    qc.cx(0,1)
    qc.unitary(np.matrix(controlled_W(W,0,2).A).getH(),[0,1,2])
    
    qc.barrier()
    
    # $CCU_{3}$ gate.
    W = sqrtm(U3(angle_theta(k_x_p,k_y_p),-angle_phi(k_x_p,k_y_p),angle_phi(k_x_p,k_y_p)))
    qc.unitary(controlled_W(W,1,2).A,[0,1,2])
    qc.cx(0,1)
    qc.unitary(np.matrix(controlled_W(W,1,2).A).getH(),[0,1,2])
    qc.cx(0,1)
    qc.unitary(np.matrix(controlled_W(W,0,2).A).getH(),[0,1,2])    
    qc.ccx(0,2,1)    
    qc.measure(0,0)
    return qc

In [14]:
#intraband_evolution(k_x,k_y,k_x_p,k_y_p).draw()

# Interband evolution circuit

In [15]:
def interband_evolution(k_x,k_y,k_x_p,k_y_p):

    # Create the quantum circuit
    qc = QuantumCircuit(3,1)  
    qc.h(0)
    qc = qc.compose(upper_band_wavefunction(k_x,k_y))    
    
    qc.barrier()
    
    qc.ccx(0,2,1)
    # $CCU_{3}^{\dagger}$ gate.
    W = sqrtm(np.matrix(U3(angle_theta(k_x,k_y),-angle_phi(k_x,k_y),angle_phi(k_x,k_y))).getH())
    qc.unitary(controlled_W(W,1,2).A,[0,1,2])
    qc.cx(0,1)
    qc.unitary(np.matrix(controlled_W(W,1,2).A).getH(),[0,1,2])
    qc.cx(0,1)
    qc.unitary(np.matrix(controlled_W(W,0,2).A).getH(),[0,1,2])
    
    qc.ccx(0,1,2)
    
    # $CCU_{3}$ gate.
    W = sqrtm(U3(angle_theta(k_x_p,k_y_p),-angle_phi(k_x_p,k_y_p),angle_phi(k_x_p,k_y_p)))
    qc.unitary(controlled_W(W,1,2).A,[0,1,2])
    qc.cx(0,1)
    qc.unitary(np.matrix(controlled_W(W,1,2).A).getH(),[0,1,2])
    qc.cx(0,1)
    qc.unitary(np.matrix(controlled_W(W,0,2).A).getH(),[0,1,2])    
    qc.ccx(0,2,1)    
    qc.measure(0,0)
    return qc

In [16]:
#interband_evolution(1,1,1,1).draw()

# Code on simulator

In [17]:
counts_lst = []
for k_1 in k_x_lst:
    for k_2 in k_y_lst:
        F_k_lst_of_points = [[(k_1,k_2),(k_1+delta_k_x,k_2)],
        [(k_1+delta_k_x,k_2),(k_1+delta_k_x,k_2+delta_k_y)],
        [(k_1,k_2+delta_k_y),(k_1+delta_k_x,k_2+delta_k_y)],
        [(k_1,k_2),(k_1,k_2+delta_k_y)]]

        for F_k_index in range(4):
            k_x,k_y = F_k_lst_of_points[F_k_index][0][0],F_k_lst_of_points[F_k_index][0][1]
            k_x_p,k_y_p = F_k_lst_of_points[F_k_index][1][0],F_k_lst_of_points[F_k_index][1][1]
            # Simulating the Circuit:
            backend = Aer.get_backend('qasm_simulator')
            job = execute(intraband_evolution(k_x,k_y,k_x_p,k_y_p), backend, shots = 5120)
            result = job.result()
            counts = result.get_counts()
            counts_lst.append([k_x,k_y,k_x_p,k_y_p,counts["0"],counts["1"]])
            #plot_histogram(counts)

In [18]:
#counts_lst

In [19]:
intraband_evolution(k_x,k_y,k_x_p,k_y_p).depth()

17

# Code on hardware

In [20]:
# Load IBM Q account and get the least busy backend device
provider = IBMQ.load_account()
provider = IBMQ.get_provider("ibm-q")
device = least_busy(provider.backends(filters=lambda x: int(x.configuration().n_qubits) >= 3 and 
                                   not x.configuration().simulator and x.status().operational==True))
print("Running on current least busy device: ", device)

ibmqfactory.load_account:WARNING:2023-10-02 14:41:19,637: Credentials are already in use. The existing account in the session will be replaced.


Running on current least busy device:  ibm_nairobi


In [21]:
# Run our circuit on the least busy backend. Monitor the execution of the job in the queue
from qiskit.tools.monitor import job_monitor
real_hardware_counts_lst = []
for k_1 in k_x_lst:
    for k_2 in k_y_lst:
        F_k_lst_of_points = [[(k_1,k_2),(k_1+delta_k_x,k_2)],
        [(k_1+delta_k_x,k_2),(k_1+delta_k_x,k_2+delta_k_y)],
        [(k_1,k_2+delta_k_y),(k_1+delta_k_x,k_2+delta_k_y)],
        [(k_1,k_2),(k_1,k_2+delta_k_y)]]

        for F_k_index in range(4):
            k_x,k_y = F_k_lst_of_points[F_k_index][0][0],F_k_lst_of_points[F_k_index][0][1]
            k_x_p,k_y_p = F_k_lst_of_points[F_k_index][1][0],F_k_lst_of_points[F_k_index][1][1]
            transpiled_circuit = transpile(intraband_evolution(k_x,k_y,k_x_p,k_y_p), device, optimization_level=3)
            job = device.run(transpiled_circuit)
            job_monitor(job, interval=2)
            # Getting the results from the computation:
            results = job.result()
            answer = results.get_counts(transpiled_circuit)
            #plot_histogram(answer)
            real_hardware_counts_lst.append([k_x,k_y,k_x_p,k_y_p,answer["0"],answer["1"]])

Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
J

In [24]:
real_hardware_counts_lst

[[-3.141592653589793,
  -3.141592653589793,
  -1.0471975511965979,
  -3.141592653589793,
  2311,
  1689],
 [-1.0471975511965979,
  -3.141592653589793,
  -1.0471975511965979,
  -1.0471975511965979,
  2146,
  1854],
 [-3.141592653589793,
  -1.0471975511965979,
  -1.0471975511965979,
  -1.0471975511965979,
  2311,
  1689],
 [-3.141592653589793,
  -3.141592653589793,
  -3.141592653589793,
  -1.0471975511965979,
  2260,
  1740],
 [-3.141592653589793,
  -1.0471975511965979,
  -1.0471975511965979,
  -1.0471975511965979,
  2180,
  1820],
 [-1.0471975511965979,
  -1.0471975511965979,
  -1.0471975511965979,
  1.0471975511965974,
  2087,
  1913],
 [-3.141592653589793,
  1.0471975511965974,
  -1.0471975511965979,
  1.0471975511965974,
  2292,
  1708],
 [-3.141592653589793,
  -1.0471975511965979,
  -3.141592653589793,
  1.0471975511965974,
  2125,
  1875],
 [-3.141592653589793,
  1.0471975511965974,
  -1.0471975511965979,
  1.0471975511965974,
  2198,
  1802],
 [-1.0471975511965979,
  1.04719755119

In [23]:
# Getting the results from the computation:
#results = job.result()
#answer = results.get_counts(transpiled_circuit)
#plot_histogram(answer)